In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from tqdm import tqdm

from util import load_data, separate_data
from models.graphcnn import GraphCNN

In [18]:
args = {
    'dataset': 'MUTAG',
    'device': 0,
    'batch_size': 32,
    'iters_per_epoch': 50,
    'epochs': 5,
    'lr': 0.01,
    'seed': 0,
    'fold_idx': 0,
    'num_layers': 5,
    'num_mlp_layers': 2,
    'hidden_dim': 64,
    'final_dropout': 0.5,
    'graph_pooling_type': 'sum',
    'neighbor_pooling_type': 'sum',
    'learn_eps': False,
    'degree_as_tag': False,
    'filename': ""
}

In [19]:
def train(args, model, device, train_graphs, optimizer, epoch):
    model.train()

    total_iters = args['iters_per_epoch']
    pbar = tqdm(range(total_iters), unit='batch')

    loss_accum = 0
    for pos in pbar:
        selected_idx = np.random.permutation(len(train_graphs))[:args['batch_size']]

        batch_graph = [train_graphs[idx] for idx in selected_idx]
        output = model(batch_graph)

        labels = torch.LongTensor([graph.label for graph in batch_graph]).to(device)

        #compute loss
        loss = criterion(output, labels)

        #backprop
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()         
            optimizer.step()
        

        loss = loss.detach().cpu().numpy()
        loss_accum += loss

        #report
        pbar.set_description('epoch: %d' % (epoch))

    average_loss = loss_accum/total_iters
    print("loss training: %f" % (average_loss))
    
    return average_loss

###pass data to model with minibatch during testing to avoid memory overflow (does not perform backpropagation)
def pass_data_iteratively(model, graphs, minibatch_size = 64):
    model.eval()
    output = []
    idx = np.arange(len(graphs))
    for i in range(0, len(graphs), minibatch_size):
        sampled_idx = idx[i:i+minibatch_size]
        if len(sampled_idx) == 0:
            continue
        output.append(model([graphs[j] for j in sampled_idx]).detach())
    return torch.cat(output, 0)

def test(args, model, device, train_graphs, test_graphs, epoch):
    model.eval()

    output = pass_data_iteratively(model, train_graphs)
    pred = output.max(1, keepdim=True)[1]
    labels = torch.LongTensor([graph.label for graph in train_graphs]).to(device)
    correct = pred.eq(labels.view_as(pred)).sum().cpu().item()
    acc_train = correct / float(len(train_graphs))

    output = pass_data_iteratively(model, test_graphs)
    pred = output.max(1, keepdim=True)[1]
    labels = torch.LongTensor([graph.label for graph in test_graphs]).to(device)
    correct = pred.eq(labels.view_as(pred)).sum().cpu().item()
    acc_test = correct / float(len(test_graphs))

    print("accuracy train: %f test: %f" % (acc_train, acc_test))

    return acc_train, acc_test

In [31]:
class FeatsGraphCNN(GraphCNN):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def __preprocess_neighbors_maxpool(self, batch_graph):
        ###create padded_neighbor_list in concatenated graph

        #compute the maximum number of neighbors within the graphs in the current minibatch
        max_deg = max([graph.max_neighbor for graph in batch_graph])

        padded_neighbor_list = []
        start_idx = [0]


        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            padded_neighbors = []
            for j in range(len(graph.neighbors)):
                #add off-set values to the neighbor indices
                pad = [n + start_idx[i] for n in graph.neighbors[j]]
                #padding, dummy data is assumed to be stored in -1
                pad.extend([-1]*(max_deg - len(pad)))

                #Add center nodes in the maxpooling if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.
                if not self.learn_eps:
                    pad.append(j + start_idx[i])

                padded_neighbors.append(pad)
            padded_neighbor_list.extend(padded_neighbors)

        return torch.LongTensor(padded_neighbor_list)


    def __preprocess_neighbors_sumavepool(self, batch_graph):
        ###create block diagonal sparse matrix

        edge_mat_list = []
        start_idx = [0]
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            edge_mat_list.append(graph.edge_mat + start_idx[i])
        Adj_block_idx = torch.cat(edge_mat_list, 1)
        Adj_block_elem = torch.ones(Adj_block_idx.shape[1])

        #Add self-loops in the adjacency matrix if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.

        if not self.learn_eps:
            num_node = start_idx[-1]
            self_loop_edge = torch.LongTensor([range(num_node), range(num_node)])
            elem = torch.ones(num_node)
            Adj_block_idx = torch.cat([Adj_block_idx, self_loop_edge], 1)
            Adj_block_elem = torch.cat([Adj_block_elem, elem], 0)

        Adj_block = torch.sparse.FloatTensor(Adj_block_idx, Adj_block_elem, torch.Size([start_idx[-1],start_idx[-1]]))

        return Adj_block.to(self.device)


    def __preprocess_graphpool(self, batch_graph):
        ###create sum or average pooling sparse matrix over entire nodes in each graph (num graphs x num nodes)
        
        start_idx = [0]

        #compute the padded neighbor list
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))

        idx = []
        elem = []
        for i, graph in enumerate(batch_graph):
            ###average pooling
            if self.graph_pooling_type == "average":
                elem.extend([1./len(graph.g)]*len(graph.g))
            
            else:
            ###sum pooling
                elem.extend([1]*len(graph.g))

            idx.extend([[i, j] for j in range(start_idx[i], start_idx[i+1], 1)])
        elem = torch.FloatTensor(elem)
        idx = torch.LongTensor(idx).transpose(0,1)
        graph_pool = torch.sparse.FloatTensor(idx, elem, torch.Size([len(batch_graph), start_idx[-1]]))
        
        return graph_pool.to(self.device)
    
    def features(self, batch_graph):
        X_concat = torch.cat([graph.node_features for graph in batch_graph], 0).to(self.device)
        graph_pool = self.__preprocess_graphpool(batch_graph)

        if self.neighbor_pooling_type == "max":
            padded_neighbor_list = self.__preprocess_neighbors_maxpool(batch_graph)
        else:
            Adj_block = self.__preprocess_neighbors_sumavepool(batch_graph)

        #list of hidden representation at each layer (including input)
        hidden_rep = [X_concat]
        h = X_concat

        for layer in range(self.num_layers-1):
            if self.neighbor_pooling_type == "max" and self.learn_eps:
                h = self.next_layer_eps(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and self.learn_eps:
                h = self.next_layer_eps(h, layer, Adj_block = Adj_block)
            elif self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, Adj_block = Adj_block)

            hidden_rep.append(h)
        
        pooleds = [graph_pool]
        #perform pooling over all nodes in each graph in every layer
        for layer, h in enumerate(hidden_rep):
            pooled_h = torch.spmm(graph_pool, h)
            pooleds.append(pooled_h)
            
        return hidden_rep, pooleds

In [32]:
criterion = nn.CrossEntropyLoss()

#set up seeds and gpu device
torch.manual_seed(0)
np.random.seed(0)    
device = torch.device("cuda:" + str(args['device'])) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

graphs, num_classes = load_data(args['dataset'], args['degree_as_tag'])

##10-fold cross validation. Conduct an experiment on the fold specified by args.fold_idx.
train_graphs, test_graphs = separate_data(graphs, args['seed'], args['fold_idx'])

model = FeatsGraphCNN(args['num_layers'], args['num_mlp_layers'], train_graphs[0].node_features.shape[1], args['hidden_dim'], num_classes, args['final_dropout'], args['learn_eps'], args['graph_pooling_type'], args['neighbor_pooling_type'], device).to(device)

optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

loading data
# classes: 2
# maximum node tag: 7
# data: 188


In [33]:
for epoch in range(1, args['epochs'] + 1):
    scheduler.step()

    avg_loss = train(args, model, device, train_graphs, optimizer, epoch)
    acc_train, acc_test = test(args, model, device, train_graphs, test_graphs, epoch)

    if not args['filename'] == "":
        with open(args['filename'], 'w') as f:
            f.write("%f %f %f" % (avg_loss, acc_train, acc_test))
            f.write("\n")
    print("")

    print(model.eps)

/home/wgar/anaconda3/envs/geo-const/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
epoch: 2:   6%|▌         | 3/50 [00:00<00:01, 26.27batch/s]

loss training: 2.456600
accuracy train: 0.702381 test: 0.650000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 3:   6%|▌         | 3/50 [00:00<00:01, 25.06batch/s]

loss training: 0.906911
accuracy train: 0.761905 test: 0.750000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 4:   6%|▌         | 3/50 [00:00<00:01, 24.53batch/s]

loss training: 0.738349
accuracy train: 0.821429 test: 0.800000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 5:   6%|▌         | 3/50 [00:00<00:01, 26.25batch/s]

loss training: 0.969491
accuracy train: 0.559524 test: 0.500000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 5: 100%|██████████| 50/50 [00:01<00:00, 25.29batch/s]


loss training: 0.573871
accuracy train: 0.779762 test: 0.700000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


In [36]:
selected_idx = np.random.permutation(len(train_graphs))[:args['batch_size']]

batch_graph = [train_graphs[idx] for idx in selected_idx]
hiddens, pooleds = model.features(batch_graph)

In [41]:
for h in hiddens:
    print(h.shape)

torch.Size([624, 7])
torch.Size([624, 64])
torch.Size([624, 64])
torch.Size([624, 64])
torch.Size([624, 64])


In [42]:
for p in pooleds:
    print(p.shape)

torch.Size([32, 624])
torch.Size([32, 7])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 64])


In [46]:
v = pooleds[2]
v.shape

torch.Size([32, 64])

In [207]:
n, h = v.shape

In [208]:
v.unsqueeze(2).repeat((1, 1, h)).shape

torch.Size([32, 64, 64])

In [209]:
inter_feature_distance_matrix = (v.unsqueeze(2).repeat((1, 1, h)) - v.unsqueeze(1)).abs()

In [210]:
 (v.unsqueeze(0).repeat((n, 1, 1)) - v.unsqueeze(1)).sum(dim=2).abs()[:4, :4]

tensor([[  0.0000,  43.5648,  46.2721, 161.8035],
        [ 43.5648,   0.0000,   2.7072, 118.2386],
        [ 46.2721,   2.7072,   0.0000, 115.5314],
        [161.8035, 118.2386, 115.5314,   0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [211]:
torch.sum(v[0] - v[0]), torch.sum(v[0] - v[1]), torch.sum(v[0] - v[2]), torch.sum(v[0] - v[2])

(tensor(0., device='cuda:0', grad_fn=<SumBackward0>),
 tensor(-43.5648, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(-46.2721, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(-46.2721, device='cuda:0', grad_fn=<SumBackward0>))

In [212]:
inter_batch_distance_matrix =  (v.unsqueeze(0).repeat((n, 1, 1)) - v.unsqueeze(1)).sum(dim=2).abs()

In [213]:
dist_stacked = torch.cat([inter_batch_distance_matrix ** 2, torch.ones((1, n)).cuda()], dim=0)

In [214]:
dist_stacked = torch.cat([dist_stacked, torch.ones((n+1, 1)).cuda()], dim=1)

In [215]:
dist_stacked[-4:, -4:]

tensor([[0.0000e+00, 6.5267e+02, 8.1258e+03, 1.0000e+00],
        [6.5267e+02, 0.0000e+00, 1.3384e+04, 1.0000e+00],
        [8.1258e+03, 1.3384e+04, 0.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [216]:
dist_stacked[n, n] = 0.

In [217]:
dist_stacked[-4:, -4:]

tensor([[0.0000e+00, 6.5267e+02, 8.1258e+03, 1.0000e+00],
        [6.5267e+02, 0.0000e+00, 1.3384e+04, 1.0000e+00],
        [8.1258e+03, 1.3384e+04, 0.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [224]:
CM = dist_stacked.det()

In [225]:
CM

tensor(-0., device='cuda:0', grad_fn=<DetBackward>)

In [226]:
simplex_volume = (-1 ** (n + 1)) / ((np.math.factorial(n) ** 2) * (2 ** n)) * CM

In [227]:
simplex_volume

tensor(0., device='cuda:0', grad_fn=<MulBackward0>)

In [228]:
def inter_batch_distances(M: torch.tensor):
    n, h = M.shape
    return (M.unsqueeze(0).repeat((n, 1, 1)) - M.unsqueeze(1)).sum(dim=2).abs()

In [229]:
def cayley_menger_determinant(D: torch.tensor):
    n = D.shape[0]
    dist_stacked = torch.cat([D ** 2, torch.ones((1, n)).cuda()], dim=0)
    dist_stacked = torch.cat([dist_stacked, torch.ones((n+1, 1)).cuda()], dim=1)
    dist_stacked[n, n] = 0.
    CM = dist_stacked.det()
    return CM

In [230]:
def simplex_volume(n: int, CM: torch.tensor):
    return (-1 ** (n + 1)) / ((np.math.factorial(n) ** 2) * (2 ** n)) * CM

In [420]:
n = 16
print(n)
selected_idx = np.random.permutation(len(train_graphs))[:n]

batch_graph = [train_graphs[idx] for idx in selected_idx]
labels = torch.LongTensor([graph.label for graph in batch_graph]).to(device)

len_a = len((labels == 0).nonzero().squeeze(1).detach().cpu().numpy()) > 0
len_b = len((labels == 1).nonzero().squeeze(1).detach().cpu().numpy()) > 0
class_a_dict = {'graph': [batch_graph[idx] for idx in (labels == 0).nonzero().squeeze(1)], 'name': '0'} if len_a else None
class_b_dict = {'graph': [batch_graph[idx] for idx in (labels == 1).nonzero().squeeze(1)], 'name': '1'} if len_b else None

for class_dict in (class_a_dict, class_b_dict):
    if class_dict is None:
        continue
    print(class_dict['name'])
    hiddens, pooleds = model.features(class_dict['graph'])
    n = len(class_dict['graph'])
    for pooled in pooleds[1:]:
        print(simplex_volume(n, cayley_menger_determinant(inter_batch_distances(pooled))))

16
0
tensor(0., device='cuda:0')
tensor(6.1798e-23, device='cuda:0', grad_fn=<MulBackward0>)
tensor(-3.3556e-20, device='cuda:0', grad_fn=<MulBackward0>)
tensor(-8.9346e-22, device='cuda:0', grad_fn=<MulBackward0>)
tensor(7.5531e-22, device='cuda:0', grad_fn=<MulBackward0>)
1
tensor(0., device='cuda:0')
tensor(1.0217e-25, device='cuda:0', grad_fn=<MulBackward0>)
tensor(6.3180e-27, device='cuda:0', grad_fn=<MulBackward0>)
tensor(-2.6217e-30, device='cuda:0', grad_fn=<MulBackward0>)
tensor(-4.0057e-35, device='cuda:0', grad_fn=<MulBackward0>)


In [410]:
(labels == 0).nonzero().squeeze(1).shape

torch.Size([10])